In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Utility
import os
import glob
import random
# Hyperparameter tuning
import optuna
# Output control
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Computer vision
from PIL import Image, ImageOps
# Data science
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import umap
import umap.plot
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix, roc_auc_score, matthews_corrcoef, precision_score, \
    balanced_accuracy_score, f1_score, recall_score,fbeta_score,accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import SVC
# Image processing
import cv2
from skimage import io, color, measure
import skimage.filters as skifilters
# Deep learning
import torch
import torch.nn as nn 
import torchvision.models as models
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pytorch_lightning as pl
import torch.optim as optim
# Consider aesthetics 
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['axes.axisbelow'] = True
plt.rcParams.update({'font.size': 20})
print("Libraries imported")

Libraries imported


In [2]:
targets = ["H1","H2","H3","H4","H5","H6"]
dataset = ["test","train","valid"]
test_dict = {}
train_dict = {}
val_dict = {}
for target,dataset in targets:
    test_dict[class_] = glob.glob(f"/kaggle/input/microscopic-fungi-images/test/{target}/*")
    
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 train_dict = {}                                                                             │
│    5 val_dict = {}                                                                               │
│    6 for target,dataset in targets:                                                              │
│ ❱  7 │   test_dict[class_] = glob.glob(f"/kaggle/input/microscopic-fungi-images/test/{target}    │
│    8                                                                                             │
│    9                                                                                             │
│   10                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'class_' is not defined

In [ ]:
test_dict